In [46]:
import numpy as np
import polars as pl
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec

# 1. Выбрать dataset с предложениями на русском языке

In [2]:
df = pl.read_csv(
    "https://github.com/0xFEE1DEAD/tatoeba_rus_to_eng/raw/refs/heads/main/tatoeba_rus_to_eng.tsv",
    separator="\t",
    has_header=False,
    new_columns=["ru_id", "ru", "eng_id", "eng"],
    encoding="utf8",
    quote_char=None,
    ignore_errors=True,
    truncate_ragged_lines=True,
    n_rows=15000,
)

In [5]:
nlp = spacy.load("ru_core_news_sm")

In [39]:
def prepare_sentence(sentence: str) -> list[str]:
    """tokenize, transform to lowercase and get lemmas for sentence."""
    prepared_sentence = nlp(sentence)
    return [token.lemma_.lower() for token in prepared_sentence if not token.is_stop and token.is_alpha]


df_with_prepared_tokens = df.with_columns(
    pl.col("ru").map_elements(prepare_sentence, return_dtype=pl.List(pl.String)).alias("tokens"),
).filter(pl.col("tokens").list.len() >= 3)

In [40]:
df_with_prepared_tokens

ru_id,ru,eng_id,eng,tokens
i64,str,i64,str,list[str]
243,"""Один раз в жизни я делаю хорош…",3257,"""For once in my life I'm doing …","[""раз"", ""жизнь"", … ""бесполезный""]"
5413,"""Сегодня 18 июня, и это день ро…",1280,"""Today is June 18th and it is M…","[""сегодня"", ""июнь"", … ""мюриэл""]"
5414,"""С днём рождения, Мюриэл!""",355041,"""Happy birthday, Muiriel!""","[""днём"", ""рождение"", ""мюриэл""]"
5426,"""Микрофон по какой-то причине д…",328546,"""For some reason the microphone…","[""микрофон"", ""причина"", ""работать""]"
5429,"""Учиться нельзя заставлять. Учё…",328883,"""Learning should not be forced.…","[""учиться"", ""заставлять"", … ""поощрять""]"
…,…,…,…,…
615070,"""Как быстро он умеет бегать!""",292312,"""How fast he can run!""","[""быстро"", ""уметь"", ""бегать""]"
615393,"""Химическая формула воды - H-O-…",270773,"""The chemical formula for water…","[""химический"", ""формула"", … ""o""]"
615398,"""Сахара - самая большая в мире …",54059,"""The Sahara is the largest dese…","[""сахара"", ""больший"", … ""пустыня""]"


Обучите модель TF-IDF и векторизуйте предложения обучающего датасета

In [41]:
vectorizer = TfidfVectorizer(lowercase=False, preprocessor=lambda x: x, tokenizer=lambda x: x, token_pattern=None)

In [42]:
tfidf_matrix = vectorizer.fit_transform(df_with_prepared_tokens["tokens"].to_list())

# Возьмите несколько предложений из тестового датасета и найдите для них топ-3 наиболее близких предложения из обучающего датасета

In [43]:
for sentence_idx in [3, 6, 9, 15, 30]:
    print("Запрос: " + "=" * 10)
    print(df_with_prepared_tokens[sentence_idx]["ru"].to_list()[0])
    tokens = df_with_prepared_tokens[sentence_idx]["tokens"].to_list()
    query_vec = vectorizer.transform(tokens)
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_3_indices = np.argsort(similarities)[-3:][::-1]
    print("Результат (топ 3): ")
    print(df_with_prepared_tokens[top_3_indices]["ru"].to_list())
    print("=" * 18)

Запрос: ==========
Микрофон по какой-то причине до этого не работал.
Результат (топ 3): 
['Микрофон по какой-то причине до этого не работал.', 'Это была причина, по которой он поступил в университет.', 'Он использует любую причину, чтобы уйти с работы.']
Запрос: ==========
Я зарабатываю сто евро в день.
Результат (топ 3): 
['Я зарабатываю сто евро в день.', 'Я зарабатываю 100 евро в день.', 'Мой муж зарабатывает 100 000 евро в год.']
Запрос: ==========
Я полагаю, это будет выглядеть несколько иначе, когда ты подумаешь об этом в долгосрочном плане.
Результат (топ 3): 
['Я полагаю, это будет выглядеть несколько иначе, когда ты подумаешь об этом в долгосрочном плане.', 'Твой план выглядит лучше моего.', 'Надо полагать, он брат Мэри.']
Запрос: ==========
Если бы мир не был таким, каков он сейчас, я мог бы доверять людям.
Результат (топ 3): 
['Если бы мир не был таким, каков он сейчас, я мог бы доверять людям.', 'Я доверяю Ричарду. Он человек слова.', 'Вам остаётся только доверять друг друг

# Word2vec

In [60]:
def prepare_sentence_word2vec(sentence: str) -> list[str]:
    """tokenize, transform to lowercase."""
    prepared_sentence = nlp(sentence)
    return [token.text.lower() for token in prepared_sentence]


df_with_prepared_tokens_w2v = df.with_columns(
    pl.col("ru").map_elements(prepare_sentence_word2vec, return_dtype=pl.List(pl.String)).alias("tokens"),
).filter(pl.col("tokens").list.len() >= 7)

In [61]:
df_with_prepared_tokens_w2v

ru_id,ru,eng_id,eng,tokens
i64,str,i64,str,list[str]
243,"""Один раз в жизни я делаю хорош…",3257,"""For once in my life I'm doing …","[""один"", ""раз"", … "".""]"
5413,"""Сегодня 18 июня, и это день ро…",1280,"""Today is June 18th and it is M…","[""сегодня"", ""18"", … ""!""]"
5421,"""Не знаю, что и сказать.""",1288,"""I just don't know what to say.""","[""не"", ""знаю"", … "".""]"
5425,"""Я не знаю, будет ли у меня вре…",531184,"""I don't know if I'll have time…","[""я"", ""не"", … "".""]"
5426,"""Микрофон по какой-то причине д…",328546,"""For some reason the microphone…","[""микрофон"", ""по"", … "".""]"
…,…,…,…,…
614594,"""Я отвечу тебе, когда ты успоко…",1184070,"""I'll answer you when you've ca…","[""я"", ""отвечу"", … "".""]"
615393,"""Химическая формула воды - H-O-…",270773,"""The chemical formula for water…","[""химическая"", ""формула"", … ""h.""]"
615398,"""Сахара - самая большая в мире …",54059,"""The Sahara is the largest dese…","[""сахара"", ""-"", … "".""]"


In [62]:
w2v_model = Word2Vec(
    sentences=df_with_prepared_tokens_w2v["tokens"].to_list(),
    vector_size=100,
    window=5,
    min_count=3,
    workers=4,
    sg=1,
)

In [63]:
model = w2v_model.wv

In [64]:
def sentence_to_vec(tokens: list[str]) -> np.ndarray:
    """Get vectors for tokens."""
    vectors = [model[word] for word in tokens if word in model]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

In [65]:
df_with_prepared_tokens_w2v = df_with_prepared_tokens_w2v.with_columns(
    pl.col("tokens").map_elements(sentence_to_vec, return_dtype=pl.List(pl.Float32)).alias("vectors"),
)

In [66]:
df_with_prepared_tokens_w2v

ru_id,ru,eng_id,eng,tokens,vectors
i64,str,i64,str,list[str],list[f32]
243,"""Один раз в жизни я делаю хорош…",3257,"""For once in my life I'm doing …","[""один"", ""раз"", … "".""]","[-0.088199, 0.171207, … -0.00736]"
5413,"""Сегодня 18 июня, и это день ро…",1280,"""Today is June 18th and it is M…","[""сегодня"", ""18"", … ""!""]","[-0.103766, 0.147823, … 0.018677]"
5421,"""Не знаю, что и сказать.""",1288,"""I just don't know what to say.""","[""не"", ""знаю"", … "".""]","[-0.045393, 0.186346, … 0.077163]"
5425,"""Я не знаю, будет ли у меня вре…",531184,"""I don't know if I'll have time…","[""я"", ""не"", … "".""]","[-0.060721, 0.116041, … 0.039146]"
5426,"""Микрофон по какой-то причине д…",328546,"""For some reason the microphone…","[""микрофон"", ""по"", … "".""]","[0.008423, 0.189161, … 0.028939]"
…,…,…,…,…,…
614594,"""Я отвечу тебе, когда ты успоко…",1184070,"""I'll answer you when you've ca…","[""я"", ""отвечу"", … "".""]","[-0.067618, 0.193644, … 0.081355]"
615393,"""Химическая формула воды - H-O-…",270773,"""The chemical formula for water…","[""химическая"", ""формула"", … ""h.""]","[0.121887, 0.205101, … -0.026129]"
615398,"""Сахара - самая большая в мире …",54059,"""The Sahara is the largest dese…","[""сахара"", ""-"", … "".""]","[-0.035681, 0.172305, … -0.0266]"


In [67]:
all_vectors = df_with_prepared_tokens_w2v["vectors"].to_list()

for sentence_idx in [3, 6, 9, 15, 30]:
    print("Запрос: " + "=" * 10)
    print(df_with_prepared_tokens_w2v[sentence_idx]["ru"].to_list()[0])
    query_vec = df_with_prepared_tokens_w2v[sentence_idx]["vectors"].to_list()
    similarities = cosine_similarity(query_vec, all_vectors).flatten()
    top_3_indices = np.argsort(similarities)[-3:][::-1]
    print("Результат (топ 3): ")
    print(df_with_prepared_tokens_w2v[top_3_indices]["ru"].to_list())
    print("=" * 18)

Запрос: ==========
Я не знаю, будет ли у меня время.
Результат (топ 3): 
['Я не знаю, будет ли у меня время.', 'Я думаю, у него всё получится.', 'Я не знаю, будет ли у меня время сделать это.']
Запрос: ==========
Учиться нельзя заставлять. Учёбу нужно поощрять.
Результат (топ 3): 
['Учиться нельзя заставлять. Учёбу нужно поощрять.', 'Извините. Беру свои слова обратно.', 'Девочка казалась смущена его грубым вопросом.']
Запрос: ==========
Я, может, скоро сдамся и вместо этого пойду вздремну.
Результат (топ 3): 
['Я, может, скоро сдамся и вместо этого пойду вздремну.', 'Девушку, которая играет в теннис, я знаю.', 'Я читаю только фрагменты новостей в Интернете, но так, общую картину себе представляю.']
Запрос: ==========
Мне всегда больше нравились таинственные персонажи.
Результат (топ 3): 
['Мне всегда больше нравились таинственные персонажи.', 'Лично мне вот этот больше нравится.', 'От этой книги тебе будет больше вреда, чем пользы.']
Запрос: ==========
Когда я вырасту, я хочу стать кор